In [1]:
# 라이브러리 호출
import numpy as np
import tensorflow as tf
import os
import warnings
import json
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [2]:
# JSON 파일 불러오기
with open('foodrecommand.json', encoding='UTF8') as file:
    data = json.load(file)
    
train_sentences = []
train_labels = []
labels = []
responses = []


for intent in data['data']:
    for pattern in intent['입력']:
        train_sentences.append(pattern)
        train_labels.append(intent['태그'])
    responses.append(intent['응답'])
    
    if intent['태그'] not in labels:
        labels.append(intent['태그'])
        
num_classes = len(labels)

In [3]:
encoder = LabelEncoder()
encoder.fit(train_labels)
training_labels = encoder.transform(train_labels)

In [4]:
word_size = 500 # 단어 집합의 크기
dim = 16 # 임베딩 후의 임베딩 벡터의 차원
max_len = 20 # 입력 시퀀스의 최대 길이
oov_token = "<OOV>" # 자연어처리 과정에서 미리 인덱싱하지 않은 단어들을 <OOV>로 인덱싱

tokenizer = Tokenizer(num_words=word_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [5]:
# 모델 생성
# 케라스 임베딩모델 정의

model = Sequential()
model.add(Embedding(word_size, dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu')) # 16개의 숨은 유닛을 가진 연결계층
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            8000      
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 6)                 102       
                                                                 
Total params: 8,646
Trainable params: 8,646
Non-trainable params: 0
______________________________________________________

In [6]:
epochs = 350
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/350
1/1 [==============================] - 1s 640ms/step - loss: 1.7917 - accuracy: 0.1818
Epoch 2/350
1/1 [==============================] - 0s 4ms/step - loss: 1.7911 - accuracy: 0.1818
Epoch 3/350
1/1 [==============================] - 0s 6ms/step - loss: 1.7906 - accuracy: 0.1818
Epoch 4/350
1/1 [==============================] - 0s 3ms/step - loss: 1.7902 - accuracy: 0.1818
Epoch 5/350
1/1 [==============================] - 0s 3ms/step - loss: 1.7898 - accuracy: 0.1818
Epoch 6/350
1/1 [==============================] - 0s 4ms/step - loss: 1.7894 - accuracy: 0.1818
Epoch 7/350
1/1 [==============================] - 0s 3ms/step - loss: 1.7891 - accuracy: 0.1818
Epoch 8/350
1/1 [==============================] - 0s 3ms/step - loss: 1.7887 - accuracy: 0.1818
Epoch 9/350
1/1 [==============================] - 0s 4ms/step - loss: 1.7884 - accuracy: 0.1818
Epoch 10/350
1/1 [==============================] - 0s 4ms/step - loss: 1.7880 - accuracy: 0.1818
Epoch 11/350
1/1 [=========

In [7]:
# 학습한 모델 저장
model.save("chat_model")

import pickle

# 학습된 Tokenizer 저장
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# 학습된 레이블 인코더 저장
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [ ]:
import json 
import colorama # 챗봇(웹)에서 사용할 텍스트 스타일 패키지(colorama) 호출
import random # 랜덤 라이브러리 호출
import pickle # 모델 저장용
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from colorama import Fore, Style, Back

colorama.init()

with open("foodrecommand.json", encoding='UTF8') as file:
    data = json.load(file)


def chatbot():
    # 학습한 모델 불러오기
    model = keras.models.load_model('chat_model')

    # Tokenizer 객체 불러오기
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # 라벨인코더 객체 불러오기
    with open('label_encoder.pickle', 'rb') as enc:
        encoder = pickle.load(enc)

    # 파라미터
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "사용자: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "종료": # 사용자가 '종료'를 입력할 경우
            break # 챗봇 종료

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = encoder.inverse_transform([np.argmax(result)])

        for i in data['data']:
            if i['태그'] == tag:
                print(Fore.GREEN + "응답:" + Style.RESET_ALL , np.random.choice(i['응답']))

print(Fore.YELLOW + "추천받고 싶은 음식 카테고리를 입력하세요. 채팅창에'종료'입력시 종료됩니다." + Style.RESET_ALL)
chatbot()

추천받고 싶은 음식 카테고리를 입력하세요. 채팅창에'종료'입력시 종료됩니다.
사용자: 

 비오는날 먹을만한 분식을 추천해주세요.


1/1 [==============================] - 0s 89ms/step
응답: 양념갈비를 추천해드려요.
사용자: 

 고기먹을거 추천해주세요


1/1 [==============================] - 0s 18ms/step
응답: 삼겹살을 추천해드려요.
사용자: 

...

1/1 [==============================] - 0s 18ms/step
응답: 신라면을 추천해드려요.
사용자: 

 간단히 먹을것좀 추천해주세요


1/1 [==============================] - 0s 19ms/step
응답: 양념갈비를 추천해드려요.
사용자: 

 간단히 먹을거 추천해주세요.


1/1 [==============================] - 0s 19ms/step
응답: 짜파게티를 추천해드려요.
사용자: 